In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf
from scipy import signal
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm import tqdm
import h5py

In [3]:
import musdb
mus = musdb.DB('~/pf/musdb18')

def stft_samples_from_audio(audio,is_vocal,n_stft_frames,rate):
    stft_samples = []
    f,t, stft_data = signal.stft(audio[:,0] + audio[:,1], rate, nperseg=1024,noverlap=512)
    if is_vocal:
        for j in range(len(stft_data.transpose())-n_stft_frames):
            stft_samples.append(stft_data[:,j+int(n_stft_frames/2)])
    else:
        for j in range(len(stft_data.transpose())-n_stft_frames):
            stft_samples.append(stft_data[:,j:j+n_stft_frames])
    return stft_samples

In [3]:
sample = stft_samples_from_audio(mus[0].audio,0,25,mus[0].rate)

In [1]:
from keras.models import model_from_json

with tf.device('/device:GPU:0'):
  freq_bins = 513
  sample_len = 25
  model = Sequential()
  model.add(Conv2D(16, (3,3), padding='same', input_shape=(freq_bins, sample_len, 1)))
  model.add(LeakyReLU())
  model.add(Conv2D(16, (3,3), padding='same'))
  model.add(LeakyReLU())
  model.add(MaxPooling2D(pool_size=(3,3)))
  model.add(Dropout(0.25))
  model.add(Conv2D(16, (3,3), padding='same'))
  model.add(LeakyReLU())
  model.add(Conv2D(16, (3,3), padding='same'))
  model.add(LeakyReLU())
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(64))
  model.add(LeakyReLU())
  model.add(Dropout(0.5))
  model.add(Dense(freq_bins, activation='relu'))
  model.compile(loss=keras.losses.mse, optimizer='adam', metrics=['mse'])
  model.load_weights("model.h5")

Using TensorFlow backend.


NameError: name 'tf' is not defined

In [6]:


a = model.predict(sample)
test = np.transpose(a)
test_list = []
for i in range(len(test)):
  test_list.append(np.roll(test[i],2))
test = np.array(test_list)


result_stft = np.multiply(test,stft_list[0])
plt.pcolormesh(t, f, np.abs(result_stft))
xmin =0
xmax = 3
ymin =0
ymax = 3000

plt.axis([xmin, xmax, ymin, ymax])


NameError: name 'model' is not defined

In [ ]:
plt.pcolormesh(t, f, np.abs(vocal_stft_list[2]))
xmin =0
xmax = 30
ymin =0
ymax = 3000

plt.axis([xmin, xmax, ymin, ymax])

In [ ]:
audio1 = signal.istft(result_stft,fs =mus[0].rate)
audio1

In [ ]:
ipd.Audio(audio1[1],rate=mus[0].rate)

In [ ]:
ipd.Audio(mus[10].audio[:,0] + mus[10].audio[:,1],rate=mus[0].rate)